# ДЗ1 по предмету "Технологии решения задач машинного обучения", Инженерия Данных, МИСИС

## Загрузка данных и разделение на трейн и тест

In [93]:
import pandas as pd
from sklearn.model_selection import train_test_split
dt = pd.read_csv('utk_vecs_train.csv')

dt_train, dt_test = train_test_split(
    dt, 
    train_size=17000,
    random_state=32
)
print(dt_train.shape, dt_test.shape)

(17000, 515) (3000, 515)


In [94]:
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import StackingRegressor, StackingClassifier
from sklearn.linear_model import Ridge, LogisticRegression, Lasso
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score

In [95]:
import warnings
warnings.filterwarnings('ignore')

## 1. Регрессия (предсказание возраста). Выбор оптимальной модели.
### Ridge (MAE - 7.11)
Обучение и оценка модели Ridge() c гиперпараметрами по умолчанию (сделано на лекции)

In [96]:

model = Ridge()

model.fit(
    dt_train.drop(['age', 'race', 'gender'], axis=1).values,
    dt_train['age'].values
)
mean_absolute_error(
    model.predict(
        dt_test.drop(['age', 'race', 'gender'], axis=1).values
    ),
    dt_test['age'].values
)

7.109253719157683

### CatBoost (MAE - 6.81)
Обучение и оценка модели CatBoost с параметрами по умолчанию (сделано на лекции)

In [97]:
model = CatBoostRegressor(verbose=False)

model.fit(
    dt_train.drop(['age', 'race', 'gender'], axis=1).values,
    dt_train['age'].values
)
mean_absolute_error(
    model.predict(
        dt_test.drop(['age', 'race', 'gender'], axis=1).values
    ),
    dt_test['age'].values
)

6.811448574687635

### Bagging (incl. CatBoost) (MAE - 6.7)
Bagging нескольких моделей, включая CatBoost (сделано на лекции) 

In [98]:
models = [
    ('ridge', Ridge()),
    ('lasso', Lasso()),
    ('catboost', CatBoostRegressor(verbose=False)),
    ('kmeans', KNeighborsRegressor())
]
predictions = []

for name, model in models:
    print(f'fitting {name}')
    model.fit(
        dt_train.drop(['age', 'race', 'gender'], axis=1).values,
        dt_train['age'].values
    )
    predictions.append(
        model.predict(
            dt_test.drop(['age', 'race', 'gender'], axis=1).values
        )
    )

predictions = sum(predictions) / len(predictions)
mean_absolute_error(predictions, dt_test['age'].values)

fitting ridge
fitting lasso
fitting catboost
fitting kmeans


6.704717085464105

### Stacking (MAE - 6.11)
Обучение и оценка модели методом стекинга

In [99]:
models = [
    ('ridge', Ridge()),
    ('lasso', Lasso()),
    # ('catboost', CatBoostRegressor(verbose=False)),
    ('kmeans', KNeighborsRegressor())
]

# meta_model = ElasticNet()  # CatBoostRegressor(verbose=True)
meta_model = CatBoostRegressor(verbose=False)

ensemble = StackingRegressor(
    estimators=models,
    final_estimator=meta_model,
    verbose=True
)


ensemble.fit(
        dt_train.drop(['age', 'race', 'gender'], axis=1).values,
        dt_train['age'].values
    )


mean_absolute_error(
    ensemble.predict(
        dt_test.drop(['age', 'race', 'gender'], axis=1).values
    ),
    dt_test['age'].values
)

6.111166533160724

In [100]:
"""Фиксируем лучшую модель"""
model_age = ensemble

## Классификация бинарная (предсказание пола). Обучение и выбор лучшей модели
### Bagging (incl. CatBoost) (accuracy - 0.9197)
Обучение и оценка моделей методом бэггинга  (сделано на занятии)

In [101]:
models = [
    ('log_reg', LogisticRegression()),
    ('catboost', CatBoostClassifier(verbose=False)),
    ('kmeans', KNeighborsClassifier())
]

predictions = []
for name, model in models:
    # print(f'fitting {name}')
    model.fit(
        dt_train.drop(['age', 'race', 'gender'], axis=1).values,
        dt_train['gender'].values
    )
    predictions.append(
        model.predict_proba(
            dt_test.drop(['age', 'race', 'gender'], axis=1).values
        )
    )
    
mean_predictions = sum(predictions) / len(predictions)
# np.argmax(mean_predictions, axis=1)
accuracy_score(np.argmax(mean_predictions, axis=1), dt_test['gender'].values)

0.9196666666666666

### Bagging (incl. CatBoost) with tuned models (accuracy - 0.9204)
Обучение моделей методом бэггинга и оценка (с предварительным подбором оптимальных гиперпараметров чере GridSearchCV)

In [102]:
models = [
    ('log_reg', LogisticRegression(C = 100, penalty = 'l2')),
    ('catboost', CatBoostClassifier(verbose=False)),
    ('kmeans', KNeighborsClassifier(n_neighbors=8, p=1))
]

predictions = []
for name, model in models:
    # print(f'fitting {name}')
    model.fit(
        dt_train.drop(['age', 'race', 'gender'], axis=1).values,
        dt_train['gender'].values
    )
    predictions.append(
        model.predict_proba(
            dt_test.drop(['age', 'race', 'gender'], axis=1).values
        )
    )
    
mean_predictions = sum(predictions) / len(predictions)
# np.argmax(mean_predictions, axis=1)
accuracy_score(np.argmax(mean_predictions, axis=1), dt_test['gender'].values)

0.9203333333333333

### Stacking (accuracy - 0.9207)
Обучение моделей стекингом. Для базовых моделей использованы значения гиперпараметров по умолчанию, т.е. при использовании тех, что подобраны через GridSearchCV качество почему-то падает. 

In [103]:
from sklearn.linear_model import RidgeClassifier

models = [
    ('log_reg', LogisticRegression()),
    ('ridge', RidgeClassifier()),
    # ('catboost', CatBoostClassifier(verbose=False)),
    ('kmeans', KNeighborsClassifier())
]

# meta_model = ElasticNet()  # CatBoostRegressor(verbose=True)
meta_model = CatBoostClassifier(verbose=False)

ensemble = StackingClassifier(
    estimators=models,
    final_estimator=meta_model,
    verbose=True
)


ensemble.fit(
        dt_train.drop(['age', 'race', 'gender'], axis=1).values,
        dt_train['gender'].values
    )

predictions = ensemble.predict(dt_test.drop(['age', 'race', 'gender'], axis=1).values)
accuracy_score(predictions, dt_test['gender'].values)

0.9206666666666666

In [104]:
"""Фиксируем лучшую модель"""
model_gender = ensemble

## Классификация мультиклассовая (предсказание расы). Обучение и выбор лучшей модели
### Bagging (incl. CatBoost) (accuracy - 0.817)
Обучение методом бэггинга (сделано на занятии)

In [105]:
models = [
    ('log_reg', LogisticRegression()),
    ('catboost', CatBoostClassifier(verbose=False)),
    ('kmeans', KNeighborsClassifier())
]

In [106]:
predictions = []

for name, model in models:
    # print(f'fitting {name}')
    model.fit(
        dt_train.drop(['age', 'race', 'gender'], axis=1).values,
        dt_train['race'].values
    )
    predictions.append(
        model.predict_proba(
            dt_test.drop(['age', 'race', 'gender'], axis=1).values
        )
    )

predictions = sum(predictions) / len(predictions)
predictions = np.argmax(predictions, axis=1)
accuracy_score(predictions, dt_test['race'].values)

0.817

### Bagging (incl. CatBoost) with tuned models (accuracy - 0.822)
Обучение методом бэггинга с использованием гиперпараметров, подобранных через GridSearchCV

In [107]:
models = [
    ('log_reg', LogisticRegression(C=0.1, penalty='l2')),
    ('catboost', CatBoostClassifier(verbose=False)),
    ('kmeans', KNeighborsClassifier(n_neighbors=10, p=2))
]
predictions = []

for name, model in models:
    # print(f'fitting {name}')
    model.fit(
        dt_train.drop(['age', 'race', 'gender'], axis=1).values,
        dt_train['race'].values
    )
    predictions.append(
        model.predict_proba(
            dt_test.drop(['age', 'race', 'gender'], axis=1).values
        )
    )

predictions = sum(predictions) / len(predictions)
predictions = np.argmax(predictions, axis=1)
accuracy_score(predictions, dt_test['race'].values)

0.822

__Фрагмент с кодом для подбора гиперараметров__

In [108]:
from sklearn.model_selection import GridSearchCV

# model = LogisticRegression()
# grid = {
#     'C': [0.1, 1.0, 10.0, 100.0],
#     'penalty' : [None, 'l1', 'l2', 'elasticnet']
# }
 
# model = RidgeClassifier()
# grid = {
#     'alpha' : np.logspace(-1, 3, 10)
# }

model = KNeighborsClassifier()
grid = {
    'n_neighbors' : np.arange(1, 11, 1),
    'p' : [1,2]
}

search = GridSearchCV(
    estimator=model,
    param_grid=grid,
    scoring='accuracy',
)

search.fit(
    dt_train.drop(['age', 'race', 'gender'], axis=1).values,
    dt_train['gender'].values
)
print(search.best_score_)
print(search.best_params_)

0.8950588235294118
{'n_neighbors': 8, 'p': 1}


### Stacking (accuracy 0.827)
Обучение модели методом стекинга. Для базовых моделей гиперпараметры подобраны с помощью GridSearchCV

In [109]:
models = [
    ('log_reg', LogisticRegression(C=0.1, penalty='l2')),
    ('ridge', RidgeClassifier(alpha=0.1)),
    # ('catboost', CatBoostClassifier(verbose=False)),
    ('kmeans', KNeighborsClassifier(n_neighbors=10, p=2))
]

# meta_model = ElasticNet()  # CatBoostRegressor(verbose=True)
meta_model = CatBoostClassifier(verbose=False)

ensemble = StackingClassifier(
    estimators=models,
    final_estimator=meta_model,
    verbose=True
)


ensemble.fit(
        dt_train.drop(['age', 'race', 'gender'], axis=1).values,
        dt_train['race'].values
    )

predictions = ensemble.predict(dt_test.drop(['age', 'race', 'gender'], axis=1).values)
accuracy_score(predictions, dt_test['race'].values)

0.8256666666666667

In [110]:
"""Фиксируем лучшую модель"""
model_race = ensemble

## Использование моделей на отдельной тестовой выборке (для сдачи на проверку)

In [113]:
# # Допустим у вас есть модель для возраста
# 
# model_age = Ridge()
# model_age.fit(np.zeros((10, 512)), np.zeros(10))
# 
# # Допустим у вас есть модель для гендера
# 
# model_gender = LogisticRegression()
# model_gender.fit(np.zeros((10, 512)), np.arange(10) % 2)
# 
# # Допустим у вас есть модель для пола
# 
# model_race = LogisticRegression()
# model_race.fit(np.zeros((10, 512)), np.arange(10) % 5)
# 
# 
# ========== Вот тут код для генерация файла предсказаний для этих моделек ============

test = pd.read_csv('utk_vecs_test_wo_target.csv')
age = model_age.predict(test.values)
gender = model_gender.predict(test.values)
race = model_race.predict(test.values)

dt = pd.DataFrame()
dt['age'] = age
dt['gender'] = gender
dt['race'] = race

# Вот этот файлик вы мне и отправляете
dt.to_csv('submission.csv', index=False)
dt

,age,gender,race
0,35.674891,0.0,3.0
1,65.083671,0.0,1.0
2,32.988188,0.0,3.0
3,24.273919,1.0,0.0
4,5.972107,0.0,0.0
...,...,...,...
3700,13.898428,0.0,2.0
3701,33.282203,0.0,0.0
3702,34.377516,0.0,1.0
3703,59.945499,0.0,0.0
